## 1.Download and Explore Dataset

In [2]:
import numpy as np 
import pandas as pd 
import json 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install folium -c conda-forge
import folium # map rendering library
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.10.1              |             py_0          59 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         112 KB

The following NEW packages will be INSTALLED:

    branca:  0.4.0-py_0  conda-forge
    folium:  0.10.1-py_0 conda-forge
    vincent: 0.4.4-py_1  conda-forge


vincent-0.4.4        | 28 KB     | ##################################### | 100% 
folium-0.10.1        | 59 KB     | #####

In [133]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)[0]
table.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


The wikipedia table has been scraped. Now, it is time to prepare the data in order to use them for the further analysis.

In [183]:
df=table.rename(columns={"Postcode": "PostalCode"})
df=df[df.Borough!='Not assigned'].reset_index(drop=True)
for r,row in enumerate(df['Borough']):
    if df.iloc[r,2]== 'Not assigned':
        df.iloc[r,2]= df.iloc[r,1]        
df1=pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighbourhood'])
k=-1
for pcode in df['PostalCode'].unique().tolist():
    n=0
    for i,row in enumerate(df['PostalCode']):
        if pcode==row:
            n=n+1
            if n==1:
                k=k+1
                j=i
                df1=df1.append(df.iloc[j], ignore_index=True)
                df1.reset_index(drop=True)                   
            else:
                df1.iloc[k,2]=df1.iloc[k,2]+", "+ df.iloc[i,2]
                df1.reset_index(drop=True)
                     

df1.head(10)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


As can be clearly seen in the dataframe above, the data have been arranged so that the rows having "Not assigned" borough cells are no longer in the table and neighboorhood names are combined seperating them with comma for those sharing the same postalcode. In addition, Borough names have been copied to the neighbourhood cells where the names were not assigned.  

In [187]:
df1.shape

(103, 3)